# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [187]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [188]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [189]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [201]:
data1 = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
data1.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [206]:
data1_pivot = data1.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

data1_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [224]:
pivot_data1 = data1.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

data2 = data1_pivot.T
data1_pivot = data1_pivot.div(data1_pivot.sum(axis=1), axis=0)
data2 = data2.div(data2.sum(axis=1), axis=0)
data2.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,0.015191,0.000000,0.000000,0.0,0.0,0.014672,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014135,0.0
200,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.012126,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.012108,0.000000,0.0
264,0.0,0.0,0.0,0.0,0.000000,0.014075,0.013618,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.017776,0.0,0.0,0.0,0.000000,0.000000,0.0
356,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.013249,0.0
412,0.0,0.0,0.0,0.0,0.013256,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.011698,0.012659,0.016086,0.0,0.0,0.0,0.000000,0.000000,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [208]:
from scipy.spatial.distance import pdist, squareform
distances = pd.DataFrame(1/(1 + squareform(pdist(data2, 'euclidean'))),
                         index=data2.index, columns=data2.index)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [211]:
similarities = list(distances[33].sort_values(ascending=False)[1:].head().index)
similarities

[60862, 27672, 6001, 79458, 33759]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [238]:
similars = data1[data1['CustomerID'].isin(similarities)]
similars.head()

,CustomerID,ProductName,Quantity
1603,1839,"Appetizer - Mini Egg Roll, Shrimp",1
1604,1839,Assorted Desserts,1
1605,1839,Bananas,1
1606,1839,Bandage - Fexible 1x3,1
1607,1839,Beef - Ground Medium,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [240]:
group = similars.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = group.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Ocean Spray - Kiwi Strawberry,47
1,Tomatoes Tear Drop,45
2,Sausage - Liver,45
3,Wine - White Cab Sauv.on,41
4,Fond - Neutral,34


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [241]:
merged = pd.merge(ranked_products, pd.DataFrame(data2.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Ocean Spray - Kiwi Strawberry,47,0.0
1,Tomatoes Tear Drop,45,0.0
2,Sausage - Liver,45,0.0
3,Wine - White Cab Sauv.on,41,0.0
4,Fond - Neutral,34,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [243]:
recommendations = {}
customers = list(data1['CustomerID'].unique())

for customer in customers: 
    similarities = list(distances[customer].sort_values(ascending=False)[1:].head().index)
    
    similars = data1[data1['CustomerID'].isin(similarities)]
    group = similars.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = group.sort_values('Quantity', ascending=False).reset_index()  
    
    merged = pd.merge(ranked_products, pd.DataFrame(data2.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [244]:
user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [247]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data2, 'cosine'))),
                         index=data2.index, columns=data2.index)

recommendations = {}
customers = list(data1['CustomerID'].unique())

for customer in customers: 
    similarities = list(distances[customer].sort_values(ascending=False)[1:].head().index)
    
    similars = data1[data1['CustomerID'].isin(similarities)]
    group = similars.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = group.sort_values('Quantity', ascending=False).reset_index()  
    
    merged = pd.merge(ranked_products, pd.DataFrame(data2.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])
user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Knife Plastic - White,Muffin - Zero Transfat,Banana Turning,Ocean Spray - Ruby Red,Crush - Cream Soda
1,200,Sauce - Demi Glace,Cookie Chocolate Chip With,"Soup - Campbells, Cream Of",Juice - Lime,"Thyme - Lemon, Fresh"
2,264,"Water - Mineral, Natural",Chicken - Wieners,Pears - Bosc,Cake - Box Window 10x10x2.5,Cream Of Tartar
3,356,Squid U5 - Thailand,Clam Nectar,"Yogurt - Blueberry, 175 Gr",Wine - Pinot Noir Latour,Garbag Bags - Black
4,412,"Salsify, Organic",Durian Fruit,Wine - Hardys Bankside Shiraz,Bread - Raisin Walnut Oval,Gatorade - Xfactor Berry
